In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Reading Data from source

In [ ]:
data_train = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2021/train.csv")
data_test = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2021/test.csv")

In [ ]:
data_train.info()

The data  is complete so we donot need to perform any imputation.

# Exploratory Data Analysis

In [ ]:
num_cols = []
obj_col= []
for col in data_train.columns:
    if(data_train[col].dtype.kind != 'O'):
        num_cols.append(col)
    else:
        obj_col.append(col)

Separating numeric and object columns

# Data Visualisation

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

for col in num_cols:
    plt.figure(figsize=(100,20))
    g = sns.histplot((data_train[col] - np.min(data_train[col])/(np.max(data_train[col]) - np.min(data_train[col]))),kde = True)
    plt.show(g)



In [ ]:
for col in obj_col:
    plt.figure(figsize=(100,20))
    g = sns.countplot(data_train[col])
    plt.show(g)

In [ ]:
plt.figure(figsize=(20,20))
g = sns.heatmap(data_train.loc[:,num_cols].corr(),annot=True)
plt.show(g)

# Data preprocessing

In [ ]:
data_train["target"] = data_train["target"].map(lambda x : np.float32(x))

In [ ]:
def train_test_split(data_train,train_split= 0.8):
    train_mask = data_train['id'].apply(lambda  x : abs(hash(str(x))) % 10000  < train_split*10000).values
    test_mask = data_train['id'].apply(lambda  x : abs(hash(str(x))) % 10000  >= train_split*10000).values
    return data_train.iloc[train_mask,0:-1],data_train.iloc[test_mask,0:-1],data_train.iloc[train_mask,-1],data_train.iloc[test_mask,-1]

In [ ]:
x_train,x_test,y_train,y_test =  train_test_split(data_train)

# Dataset Methods

In [ ]:
def train_input_function(batch_size = 128):
    ds = tf.data.Dataset.from_tensor_slices((dict(x_train) , y_train))
    ds = ds.shuffle(len(x_train)).batch(batch_size)
    return ds

def eval_input_function(batch_size = 10):
    ds = tf.data.Dataset.from_tensor_slices((dict(x_test) , y_test))
    ds = ds.batch(batch_size)
    return ds

def albation_input_function(batch_size = 10):
    ds = tf.data.Dataset.from_tensor_slices((dict(x_train[0:1000]) , y_train[0:1000]))
    ds = ds.shuffle(1000).batch(batch_size)
    return ds

# def pred_input_function(batch_size = 10):
#     ds = tf.data.Dataset.from_tensor_slices((dict(x_test) , y_test))
#     ds = ds.batch(batch_size)
#     return ds

# Defining feature columns

In [ ]:
import tensorflow as tf

In [ ]:
def generate_feature_cols(data_train):
    cols_list = []
    for col  in data_train.columns[:-1]:
        if(data_train[col].dtype.kind != "O"):
            cols_list.append(tf.feature_column.numeric_column(col))
        else:
            cols_list.append(tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list(col,data_train[col].unique())))
    return cols_list

In [ ]:
example_batch = next(iter(train_input_function()))[0]

In [ ]:
def demo(feature_column):
    feature_layer = tf.keras.layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy()[0])

In [ ]:
demo(generate_feature_cols(data_train))

# Defining Model

In [ ]:
keras_model = tf.keras.Sequential([
    tf.keras.layers.DenseFeatures(generate_feature_cols(data_train)),
    tf.keras.layers.Dense(1024,activation = "relu",kernel_regularizer = "l1_l2"),
    tf.keras.layers.Dense(1024,activation = "relu"),
    tf.keras.layers.Dense(1024,activation = "relu"),
    tf.keras.layers.Dense(1024,activation = "relu"),
    tf.keras.layers.Dense(1,activation= "softmax")
])

In [ ]:
def loss_function(true,pred):
    return t

def scheduler(epoch, lr):
    if(epoch < 10):
        return lr
    else:
        return lr/epoch
    
callbacks = tf.keras.callbacks.LearningRateScheduler(scheduler)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(actor=0.1, patience=3, verbose=1,mode='auto', min_delta=0.0001, cooldown=0, min_lr=0,)
keras_model.compile(
    optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.01),
    loss="binary_crossentropy",
    metrics=["accuracy",tf.keras.metrics.AUC()])

# Albation Experiment

In [ ]:
hostory = keras_model.fit(train_input_function(100),epochs = 150,callbacks=[reduce_lr])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
hostory.history.keys()

In [ ]:
g = sns.lineplot([x for x in range(len(hostory.history["loss"]))],hostory.history["loss"])

In [ ]:
y2 = hostory.history["loss"][0]
y1  = hostory.history["loss"][-1]
x2 = 100
x1=  0

print("Slope is {0}".format((y2-y1)/(x2-x1)))

In [ ]:
pred = keras_model.predict(dict(data_test))
ids = data_test.loc[:,["id"]].applymap(lambda x : int(x))

In [ ]:
pred_out = pd.DataFrame({"Id": ids.values.flatten(),"target": pred.flatten()})

In [ ]:
pred_out.to_csv("./submission.csv",index=False)